## Glow GWAS 1KG QC

This tutorial will only run the initial part of the 1KG data QC.  Even this first part takes several hours so re-writing the entire tutorial this way is basically infeasible with Glow (on a single server).

In [1]:
import $file.^.init.spark, spark._
import $file.^.init.paths, paths._
import $file.^.init.glow, glow._
import $file.^.init.benchmark, benchmark._
import $file.^.init.functions, functions._
import $file.^.init.{plotly => init_plotly}, init_plotly._
import sys.process._
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import io.projectglow.Glow
import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond.{init => init_plotly_js, _}
import better.files.File
import org.apache.log4j.{Level, Logger}
Logger.getLogger("io.projectglow.plink").setLevel(Level.WARN)

def timeop[T](op: String)(block: => T) = optimer("glow", op, block)

// Increase broadcast timeout from 300 to avoid "Could not execute broadcast in 300 secs."
val ss = getLocalSparkSession(
    broadcastTimeoutSeconds=14400, 
    shufflePartitions=1,
    enableUI=true
) 
import ss.implicits._
Glow.register(ss)

init_plotly_js(offline=false)

val data_dir = GWAS_TUTORIAL_DATA_DIR / "2_PS_GWAS"
val PS1_1KG_RAW_FILE = "ALL.2of4intersection.20100804.genotypes"

Loading spark-stubs


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/eczech/.cache/coursier/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.16/slf4j-log4j12-1.7.16.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/eczech/.cache/coursier/v1/https/repo1.maven.org/maven2/org/slf4j/slf4j-log4j12/1.7.25/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
20/02/07 13:21:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark UI

import $file.$           , spark._

import $file.$           , paths._

import $file.$          , glow._

import $file.$               , benchmark._

import $file.$               , functions._

import $file.$                             , init_plotly._

import sys.process._

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.functions._

import io.projectglow.Glow

import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond.{init => init_plotly_js, _}

import better.files.File

import org.apache.log4j.{Level, Logger}

defined function timeop
ss: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@34ec0bf9
import ss.implicits._

data_dir: File = /home/eczech/data/gwas/tutorial/2_PS_GWAS
PS1_1KG_RAW_FILE: String = "ALL.2of4intersection.20100804.genotypes"

### Convert 1KG to Parquet

In [6]:
val df = ss.read.format("plink")
//     .option("famDelimiter", "\t")
//     .option("bimDelimiter", "\t")
    .load(data_dir / PS1_1KG_RAW_FILE + ".bed" toString)
df.printSchema

root
 |-- contigName: string (nullable = true)
 |-- names: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- position: double (nullable = true)
 |-- start: long (nullable = true)
 |-- end: long (nullable = true)
 |-- referenceAllele: string (nullable = true)
 |-- alternateAlleles: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- genotypes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- sampleId: string (nullable = true)
 |    |    |-- calls: array (nullable = true)
 |    |    |    |-- element: integer (containsNull = true)



df: DataFrame = [contigName: string, names: array<string> ... 6 more fields]

In [7]:
timeop("ps0") {
    df.write
    .mode("overwrite")
    .parquet(data_dir / PS1_1KG_RAW_FILE + ".parquet" toString)
}

Elapsed time: 1099.5 seconds


### 1KG QC 

In [2]:
val df = ss.read.parquet(data_dir / PS1_1KG_RAW_FILE + ".parquet" toString)
df.count()

df: DataFrame = [contigName: string, names: array<string> ... 6 more fields]
res1_1: Long = 25488488L

In [9]:
df.groupBy(size($"genotypes")).count.show

+---------------+--------+
|size(genotypes)|   count|
+---------------+--------+
|            629|25488488|
+---------------+--------+



In [2]:
// To take a sample efficiently, this appears to be the best way
// (.limit will scan every paritition first)
// val df = ss.read.parquet(data_dir / PS1_1KG_RAW_FILE + ".parquet" toString)
//     .transform(d => {ss.createDataFrame(d.takeAsList(100), d.schema)})

In [3]:
// QC functions

def count(df: DataFrame) = (df.count, df.select(size(col("genotypes"))).first.getAs[Int](0)) // (n_variants, n_samples)

def filterBySampleCallRate(threshold: Double)(df: DataFrame): DataFrame = { 
    df
        // Cross join original dataset with single-row data frame containing a map like (sampleId -> QC stats)
        .crossJoin(
            df
            .selectExpr("sample_call_summary_stats(genotypes, referenceAllele, alternateAlleles) as qc")
            .selectExpr("map_from_arrays(qc.sampleId, qc) as qc")
        )
        // For each row, filter the genotypes array (which has one element per sampleId) based on QC map lookup
        .selectExpr("*", s"filter(genotypes, g -> qc[g.sampleId].callRate >= ${threshold}) as filtered_genotypes")
        // Remove intermediate fields 
        .drop("qc", "genotypes").withColumnRenamed("filtered_genotypes", "genotypes")
        // Ensure that the original dataset schema was preserved
        .transform(d => {assert(d.schema.equals(df.schema)); d})
}

// Rewrite based on https://github.com/projectglow/glow/issues/148#issuecomment-582485763

def filterByVariantCallRate(threshold: Double)(df: DataFrame): DataFrame = { 
    df
        .selectExpr("*", "call_summary_stats(genotypes) as qc")
        .filter(col("qc.callRate") >= threshold)
        .drop("qc")
        .transform(d => {assert(d.schema.equals(df.schema)); d})
}

defined function count
defined function filterBySampleCallRate
defined function filterByVariantCallRate

**Initial Implementation**

This version of the filtering methods (```filterByVariantCallRate``` in particular) was found to be suboptimal as discussed in https://github.com/projectglow/glow/issues/148#issuecomment-582485763.

Nevertheless, it is left here for posterity since it my help others avoid similar anti-patterns.

In [ ]:
// Ran for ~3 hours (with both 1 and 200 shuffle partitions) and 
// used 30g of 64g heap before being killed.
val df_qc1 = df
    .transform(filterByVariantCallRate(threshold=.8))
    .transform(filterBySampleCallRate(threshold=.8))
    .transform(filterByVariantCallRate(threshold=.98))
    .transform(filterBySampleCallRate(threshold=.98))

val ct = timeop("ps1") {
    count(df_qc1)
}
ct

**Improved Implementation**

This rewrite, that avoids generating maps of sampleId -> qc objects, turns out to be much more efficient:

In [ ]:
// Rewrite based on https://github.com/projectglow/glow/issues/148#issuecomment-582485763
def filterBySampleCallRate(threshold: Double)(df: DataFrame): DataFrame = {
  val qc = df.selectExpr("sample_call_summary_stats(genotypes, referenceAllele, alternateAlleles) as qc")
  df.crossJoin(qc)
    // For each row, filter the genotypes array (which has one element per sampleId) based on whether it passed the QC filter
    .selectExpr("*", s"""
        transform(
            filter(
                zip_with(sequence(0, size(genotypes)), genotypes, (i, g) -> (i, g)), e -> 
                qc[e.i].callRate >= $threshold
            ), 
            e -> e.g
        ) as filtered_genotypes
    """)
    // Remove intermediate fields 
    .drop("qc", "genotypes")
    .withColumnRenamed("filtered_genotypes", "genotypes")
    // Ensure that the original dataset schema was preserved
    .transform(d => {assert(d.schema.equals(df.schema)); d})
}

In [4]:
// Try the QC again:

val df_qc1 = df
    .transform(filterByVariantCallRate(threshold=.8))
    .transform(filterBySampleCallRate(threshold=.8))
    .transform(filterByVariantCallRate(threshold=.98))
    .transform(filterBySampleCallRate(threshold=.98))

val ct = timeop("ps1") {
    count(df_qc1)
}
ct

Elapsed time: 5113.1 seconds


df_qc1: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [contigName: string, names: array<string> ... 6 more fields]
ct: (Long, Int) = (8240745L, 629)
res3_2: (Long, Int) = (8240745L, 629)